# Quarterly Submission Overview Checks

This notebook will be used for comparing and contrasting quarterly cfde submissions.

The general practice will be to create a directory using the name of the quarter in the directory this notebook exists in, and put that directory name in the first code cell that sets **DB1** and **DB2**. The first db being the oldest quarter and the second db being the newest quarter.

In [ ]:
import sqlite3
import pandas as pd

DB1, DB2 = 'Y4Q2_C2M2', 'Y4Q1_C2M2'

In [ ]:
db1_conn = sqlite3.connect(f'{DB1}.db')

query = f"""
select project_local_id, count(local_id) as {DB1}_subject_count
from subject
group by project_local_id
"""
db1_subject_df = pd.read_sql_query(query,db1_conn)

In [ ]:
db2_conn = sqlite3.connect(f'{DB2}.db')

query = f"""
select project_local_id, count(local_id) as {DB2}_subject_count
from subject
group by project_local_id
"""
db2_subject_df = pd.read_sql_query(query,db2_conn)

In [ ]:
combined_quarters_df = db1_subject_df.merge(db2_subject_df,
                                   how='inner',
                                   on='project_local_id') \
                            .sort_values(by='project_local_id')

In [ ]:
query = f"""
select project_local_id, count(distinct(local_id)) as {DB1}_biosample_count
from biosample
group by project_local_id
"""
db1_biosample_df = pd.read_sql_query(query,db1_conn)

In [ ]:
query = f"""
select project_local_id, count(distinct(local_id)) as {DB2}_biosample_count
from biosample
group by project_local_id
"""
db2_biosample_df = pd.read_sql_query(query,db2_conn)

In [ ]:
combined_quarters_df = combined_quarters_df.merge(db1_biosample_df,
                                                  how='inner',
                                                  on='project_local_id') \
                                            .merge(db2_biosample_df,
                                                   how='inner',
                                                   on='project_local_id')


In [ ]:
query = f"""
select project_local_id, count(local_id) as {DB1}_file_count
from file
group by project_local_id
"""
db1_file_df = pd.read_sql_query(query,db1_conn)

In [ ]:
query = f"""
select project_local_id, count(local_id) as {DB2}_file_count
from file
group by project_local_id
"""
db2_file_df = pd.read_sql_query(query,db2_conn)


In [ ]:
combined_quarters_df = combined_quarters_df.merge(db1_file_df,
                                                  how='inner',
                                                  on='project_local_id') \
                                            .merge(db2_file_df,
                                                   how='inner',
                                                   on='project_local_id')

In [ ]:
query = f"""
select project_local_id, count(biosample_local_id) as {DB1}_biosample_disease_count
from biosample_disease bd
join biosample b on b.local_id = bd.biosample_local_id
group by project_local_id
"""
db1_biosample_disease_df = pd.read_sql_query(query,db1_conn)

In [ ]:
query = f"""
select project_local_id, count(biosample_local_id) as {DB2}_biosample_disease_count
from biosample_disease bd
join biosample b on b.local_id = bd.biosample_local_id
group by project_local_id
"""
db2_biosample_disease_df = pd.read_sql_query(query,db2_conn)

In [ ]:
combined_quarters_df = combined_quarters_df.merge(db1_biosample_disease_df,
                                                  how='inner',
                                                  on='project_local_id') \
                                            .merge(db2_biosample_disease_df,
                                                   how='inner',
                                                   on='project_local_id')

In [ ]:
combined_quarters_df['subject_count_change'] = combined_quarters_df.apply(lambda row: row[f'{DB2}_subject_count'] - row[f'{DB1}_subject_count'],axis=1)
combined_quarters_df['biosample_count_change'] = combined_quarters_df.apply(lambda row: row[f'{DB2}_biosample_count'] - row[f'{DB1}_biosample_count'],axis=1)
combined_quarters_df['file_count_change'] = combined_quarters_df.apply(lambda row: row[f'{DB2}_file_count'] - row[f'{DB1}_file_count'],axis=1)
combined_quarters_df['biosample_disease_count_change'] = combined_quarters_df.apply(lambda row: row[f'{DB2}_biosample_disease_count'] - row[f'{DB1}_biosample_disease_count'],axis=1)

combined_quarters_df

In [ ]:
change_columns = [col for col in combined_quarters_df.columns if col.endswith('change')]
studies_with_delta = combined_quarters_df.loc[(combined_quarters_df[change_columns] != 0).any(axis=1)]

In [ ]:
f"Number of studies: {studies_with_delta.shape[0]}"

In [ ]:
study_name_df = pd.read_sql_query("select * from project",db1_conn)
study_name_df = study_name_df[["local_id","name"]]

studies_with_delta = studies_with_delta.merge(study_name_df,how="left",left_on="project_local_id",right_on="local_id")
change_columns.insert(0,"local_id")
change_columns.insert(1,"name")

from IPython.display import display
display(studies_with_delta[change_columns].style.hide_index())